<hr style="border: solid 1px red; margin-bottom: 2% ">

### ARCHER SCIENTIFIC PYTHON COURSE:

# C/Fortran Interface

<hr style="border: solid 1px red; margin-bottom: -1%; ">

## Website:  http://www.archer.ac.uk 

## Helpdesk: support@archer.ac.uk

<br>

<img src="images/epsrclogo.png" style="width: 40%;">
<br>
<img src="images/nerclogo.png" style="width: 40%;">

<br>
<img src="images/craylogo.png" style="width: 40%;">

<br>
<img src="images/epcclogo.png" style="width: 40%;">

<br>
<img src="images/ediunilogo.png" style="width: 40%" > 

<br>
<br>

<img src="images/reusematerial.png" style="width: 90">
<br>
<br>

<br>
<hr class="top">

# C/Fortran Interface

<hr class="bot">
<br>

## Presenter: Kevin Stratford

#### Contributing authors:
#### Neelofer Banglawala, Arno Proeme, Kevin Stratford, Andy Turner

<br>
<br>
<br>



<br>
<hr class="top">

## Why couple python with another language?

<hr class="bot">
<br>

* Provide glue to organise complex tasks

  * Handle complex software coordination provided by Python


* Combine performance of compiled codes with flexibility of Python

  * e.g. incorporate Python analysis and visualisation into existing codebase
  * Provide flexible way to extract results from code using Python 


* Reuse code that you already have

  * Gradually introduce new functionality using Python



* <p style="font-size: 100%">More info: </p>

  * <p style="font-size: 100%"> http://docs.scipy.org/doc/numpy-dev/f2py/ </p>
  * <p style="font-size: 100%"> http://scipy-cookbook.readthedocs.org </p>
  * <p style="font-size: 100%"> http://www.f2py.com/home/ </p>

<br>
<hr class="top">

## Extension modules

<hr class="bot">
<br>

* Basic approach is to compile *extension modules*

  * Compiles native langauge source 
  * Adds a wrapper which provides the interface


* Requires

  * Appropriate compiler (e.g., `gfortran`, `cc`, `...`)
  * A clear understanding of the number of types of any arguments


* Will produce

  * Extension module (shared library `.so`) loaded at run time
  * `import` extension module as usual


* Some care may be required with compound/opaque data types
<br>
<br>

<br>
<hr class="top">

## Different approaches

<hr class="bot">
<br>

* Fortran

    * `f2py` is part of Numpy will handle external subroutines
    * Modern Fortran requires, e.g., `f90wrap`


* C (more choice)

  * `f2py` can be used (a kludge)
  * C native interface
  * Cython
  * Swig
  * ctypes/CFFI


* Various other approaches

   * Weave, Numba
   
<br>
<br>

<br>
<hr class="top">
## Fortran and python via `f2py`
<hr class="bot">
<br>

* You need to provide `f2py` with:

  * Fortran source code
  * A *signature file*: a file describing the external function and its arguments
  * (`f2py` can help you generate a signature file)


* Also need access to a Fortran compiler


* `f2py` can:

  * Create a signature file containing argument attributes (e.g., `optional`) that define the Fortran interface
  * Wrap Fortran code in an extension module that can be imported from within Python

<br>
<br>
<hr class="top">
## General recipe
<hr class="bot">
<br>

* Create a signature file 
  * Use 
  
```python
f2py <source> -m <extension_module> -h <signature>.pyf
```

* Typically the signature filename stub is the same as the source filename


* Check the signature file for correctness
  * Sequence and types of arguments to be passed from Python to Fortran function
  * Argument attributes, such as `depend`


* Produce the final extension module
  * `f2py -c <signature_file>.pyf <source_file>.f90`


* Import module into Python and use the external Fortran function!

```python
    import extension_module_name
    extension_module_name.function(args)
```

* The source filename may not be the same as the function name


<br>
<hr class="top">
## Example : `farray_sqrt.f90`
<hr class="bot">
<br>

#### Consider:
`array_sqrt()` is an external subroutine

```fortran
subroutine array_sqrt(n, a_in, a_out)
  implicit none
  integer, intent(in) :: n
  real, dimension(n), intent(in)  :: a_in
  real, dimension(n), intent(out) :: a_out
  
  integer :: i
  
  do i = 1, n
    a_out(i) = sqrt(a_in(i))
  end do
  
  return
end subroutine array_sqrt
```


<br>
<hr class="top">
## Create a signature file
<hr class="bot">
<br>

* `f2py` can try to create the signature file (`farray_sqrt.pyf`) automatically
   
   * From a terminal, issue the command:

```bash
f2py farray_sqrt.f90 -m farray -h farray_sqrt.pyf
```

* The Python extension module will be called: `farray`

   * use the `-m` option


* Signature in text file called: `farray_sqrt.pyf`
 
  * use the `-h` option
  
  * Note: will not overwrite an existing signature file:
    Use `--overwrite-signature` to overwrite.


In [1]:
# Call from within Python to save exiting notebook...

!f2py farray_sqrt.f90 -m farray -h farray_sqrt.pyf

Reading fortran codes...
	Reading file 'farray_sqrt.f90' (format:free)
Post-processing...
	Block: farray
			Block: array_sqrt
Post-processing (stage 2)...
Saving signatures to file "./farray_sqrt.pyf"


<br>
<hr style="border: solid 1px red; margin-bottom: 2% ">
## Check signature file
<hr style="border: solid 1px red; margin-bottom: -1%; ">
<br>

Attributes such as <b>optional</b>, <b>intent</b> and <b>depend</b> specify the visibility, purpose and dependencies of the arguments.


In [2]:
!cat farray_sqrt.pyf

!    -*- f90 -*-
! Note: the context of this file is case sensitive.

python module farray ! in 
    interface  ! in :farray
        subroutine array_sqrt(n,a_in,a_out) ! in :farray:farray_sqrt.f90
            integer, optional,intent(in),check(len(a_in)>=n),depend(a_in) :: n=len(a_in)
            real dimension(n),intent(in) :: a_in
            real dimension(n),intent(out),depend(n) :: a_out
        end subroutine array_sqrt
    end interface 
end python module farray

! This file was auto-generated with f2py (version:2).
! See http://cens.ioc.ee/projects/f2py2e/


Note: exact result can depend on version of `numpy`, so it is worth checking

<br>
<hr class="top">
## Compile extension module
<hr class="bot">
<br>

Once you have verified that the signature file is correct

* Use `f2py` to compile a module file that can be imported into Python

```bash
f2py -c farray_sqrt.pyf farray_sqrt.f90
```

This should produce a shared library file called: `farray.so`

In [3]:
# Run f2py command from within notebook
# If you don't want to see the output, try "msg = !f2py ..."

!f2py -c farray_sqrt.pyf farray_sqrt.f90

running build
running config_cc
unifing config_cc, config, build_clib, build_ext, build commands --compiler options
running config_fc
unifing config_fc, config, build_clib, build_ext, build commands --fcompiler options
running build_src
build_src
building extension "farray" sources
creating /var/folders/97/w5_3cn_13t39q7pwkjc6hds00000gq/T/tmpi6souv3b/src.macosx-10.6-x86_64-3.5
f2py options: []
f2py: farray_sqrt.pyf
Reading fortran codes...
	Reading file 'farray_sqrt.pyf' (format:free)
Post-processing...
	Block: farray
			Block: array_sqrt
Post-processing (stage 2)...
Building modules...
	Building module "farray"...
		Constructing wrapper function "array_sqrt"...
		  a_out = array_sqrt(a_in,[n])
	Wrote C/API module "farray" to file "/var/folders/97/w5_3cn_13t39q7pwkjc6hds00000gq/T/tmpi6souv3b/src.macosx-10.6-x86_64-3.5/farraymodule.c"
  adding '/var/folders/97/w5_3cn_13t39q7pwkjc6hds00000gq/T/tmpi6souv3b/src.macosx-10.6-x86_64-3.5/fortranobject.c' to sources.
  adding '/var/folders/97/w

In [4]:
# Check we have the farray.so
!ls

Untitled.ipynb               fibonacci.c
c_sqrt.c                     fibonacci.f90
cfortran.ipynb               iarray_sqrt.f90
farray.cpython-35m-darwin.so images
farray_sqrt.f90              my_library.c
farray_sqrt.pyf


<br>
<hr class="top">
## Call external function from Python
<hr class="bot">
<br>

In [5]:
# import the extension module
import numpy as np
from farray import array_sqrt

In [6]:
# view docsting of function (automatically produced)
array_sqrt?

In [7]:
# let's try to use the function

ain = np.array([1.0, 4.0, 9.0, 16.0, 2.0], np.double)
print(ain)
aout = array_sqrt(ain)
print(aout)

[  1.   4.   9.  16.   2.]
[ 1.          2.          3.          4.          1.41421354]


<br>
<hr class="top">
## Python and C via `ctypes`
<hr class="bot">
<br>

* Uses only python (no additional interface file or mixed-language intermediate code)
```python
    import ctypes
```

* Must load the library (`.so`) file explicitly

```python
    lib = ctypes.cdll.LoadLibrary("./my_library.so")
```


* Must specify the prototype for the C function

```python
    lib.my_c_function.restype = ctypes.c_int
    lib.my_c_function.argtypes = [ctypes.c_double]
```

<br>
<hr class="top">
## Example: C side
<hr class="bot">
<br>

Consider the simple function:
```c
int my_c_function(double val) {

    return (int) (val + 1.0);
}
```

We need to compile an extension module:


In [8]:
!gcc -c -fPIC my_library.c
!gcc -shared -o my_library.so my_library.o
!ls

Untitled.ipynb               fibonacci.f90
c_sqrt.c                     iarray_sqrt.f90
cfortran.ipynb               images
farray.cpython-35m-darwin.so my_library.c
farray_sqrt.f90              my_library.o
farray_sqrt.pyf              my_library.so
fibonacci.c


<br>
<hr class="top">
## Example: python side
<hr class="bot">
<br>

Now:



In [9]:
import ctypes

lib = ctypes.cdll.LoadLibrary("./my_library.so")

lib.my_c_function.restype = ctypes.c_int
lib.my_c_function.argtypes = [ctypes.c_double]

x = float(23)
result = lib.my_c_function(x)
print(result, type(result))

24 <class 'int'>



<br>
<hr class="top">
## `ctypes` and `numpy.ndarray`
<hr class="bot">
<br>


Consider again the square root example, this time in C:
    
```c
#include <math.h>

void array_sqrt(int n, double * a_in, double * a_out) {

  int i;

  for (i = 0; i < n; i++) {
    a_out[i] = sqrt(a_in[i]);
  }

  return;
}
```



In [5]:
!gcc -c -fPIC c_sqrt.c
!gcc -shared -o c_sqrt.so c_sqrt.o

<br>
<br>
<hr class="top">

## Using bare ctypes

<hr class="bot">
<br>


In [17]:
# We need a pointer type p_double

import ctypes

c_int = ctypes.c_int
c_double = ctypes.c_double
p_double = ctypes.POINTER(c_double)

# Define some test data

a_in = np.array([16.0, 25.0, 36.0, 49.0])
a_out = np.empty(4, np.double)


In [18]:
# Load library, declare the interface

liba = ctypes.cdll.LoadLibrary("./c_sqrt.so")

liba.array_sqrt.restype = None
liba.array_sqrt.argtypes = [c_int, p_double, p_double]

# Then we need to pass pointers to actual data
pin  =  a_in.ctypes.data_as(p_double)
pout = a_out.ctypes.data_as(p_double)

liba.array_sqrt(4, pin, pout)
print(a_out)

[4. 5. 6. 7.]


<br>
<br>
<hr class="top">

## Using `numpy.ctypeslib`

<hr class="bot">
<br>
Couldn't we just pass the numpy `ndarray` as it is? 

In [22]:
from numpy.ctypeslib import ndpointer

libb = ctypes.cdll.LoadLibrary("./c_sqrt.so")

# Describe a_in, a_out
ndarg = ndpointer(ctypes.c_double, flags = "C_CONTIGUOUS")

libb.array_sqrt.restype = None
libb.array_sqrt.argtypes = [c_int, ndarg, ndarg]

a_out = numpy.empty(4, np.double)

libb.array_sqrt(4, a_in, a_out)
print(a_out)

<class '_ctypes.PyCSimpleType'>
[4. 5. 6. 7.]


<br>
<br>
<hr class="top">

## Using numba

<hr class="bot">
<br>
TEST ME

In [25]:
from numba import jit

@jit (nopython = True)
def array_sqrt(n, ain, aout):
    
    for i in range(n):
        aout[n] = np.sqrt(ain[n])


a_out = np.empty(4, np.double)
array_sqrt(4, a_in, a_out)
print(a_out)

TypingError: Failed at nopython (nopython frontend)
[1mUntyped global name 'accident':[0m [1m[1mcannot determine Numba type of <class 'object'>[0m
[1m
File "<ipython-input-25-41215d291acb>", line 9:[0m
[1mdef array_sqrt(n, ain, aout):
    <source elided>
        
[1m    accident
[0m    [1m^[0m[0m
[0m
This is not usually a problem with Numba itself but instead often caused by
the use of unsupported features or an issue in resolving types.

To see Python/NumPy features supported by the latest release of Numba visit:
http://numba.pydata.org/numba-doc/dev/reference/pysupported.html
and
http://numba.pydata.org/numba-doc/dev/reference/numpysupported.html

For more information about typing errors and how to debug them visit:
http://numba.pydata.org/numba-doc/latest/user/troubleshoot.html#my-code-doesn-t-compile

If you think your code should work with Numba, please report the error message
and traceback, along with a minimal reproducer at:
https://github.com/numba/numba/issues/n

<br>
<hr class="top">
## Alternatives
<hr class="bot">
<br>

* <p style="font-size: 100%"><b>Native Python interface</b></p>

  * <p style="font-size: 100%">Fully-flexible and portable</p>
  * <p style="font-size: 100%">Complex and verbose</p>
  * <p style="font-size: 100%">Option if you are interfacing a large amount of code and/or have a large software development project</p>


* <p style="font-size: 100%"><b>Cython</b> : converts Python-like code into a C library which can call other C libraries</p>

  * <p style="font-size: 100%">Standard C-like Python (or Python-like C)</p>
  
  
* <p style="font-size: 100%"> <b>SWIG</b> (or <b>S</b>implified <b>W</b>rapper and <b>I</b>nterface <b>G</b>enerator) : reads header files and generates a library Python can load </p>

  * <p style="font-size: 100%">Very generic and feature-rich</p>
  * <p style="font-size: 100%">Supports multiple languages other than Python (e.g. Perl, Ruby)</p>
  

  
  
    




<br>
<hr class="top">
## Alternatives contd ...
<hr class="bot">
<br>
  
* <p style="font-size: 100%"><b>ctypes</b>, <b>cffi</b> (C Foreign Function Interface for Python) : both provide "foreign function interfaces", or lightweight APIs, for calling C libraries from within Python</p> 


  * <p style="font-size: 100%">The goal is to provide a convenient and reliable way to call compiled C code from Python using interface declarations written in C</p>
 
  
* <p style="font-size: 100%"><b>Weave</b> : includes C/C++ code within Python code and compiles it transparently</p>


* <p style="font-size: 100%"><b>Boost.python</b> : helps write C++ libraries that Python can load and use easily  </p>
  
  
* <p style="font-size: 100%"><b>PyCUDA</b> : allows you to include NVIDIA CUDA code within Python.
    You can also write C code by hand, that can be called by Python.</p>
  
  
    




<br>
<hr class="top">
## Summary
<hr class="bot">
<br>

* Calling C/Fortran allows code re-use


* Fortran/C can give better performance than Python


* `f2py` is a simple way to call Fortran code from Python


* Modern Fortran users should consider `f90wrap`


* Other options more appropriate for C

<br>
<br>

https://github.com/jameskermode/f90wrap

<br>
<br>
<br>

In [ ]:
help(ctypes)

<br>
<hr class="top">
## Exercise: `fibonacci.f90`
<hr class="bot">
<br>

Use `f2py` to create an extension module for function `fibonacci()` and test it in Python.

`fibonacci()` computes the first `n` Fibonacci numbers: 0, 1, 1, 2, 3, 5, 8, 13,...
and stores the results in the array provided.

```fortran
subroutine fibonacci(n, a_out)
  implicit none
  integer, intent(in) :: n
  real*8, dimension(n) :: a_out

  integer :: i
  
  do i = 1, n
    if (i.eq.1) then
      a_out(i) = 0.0
    else if (i.eq.2) then
      a_out(i) = 1.0
    else
      a_out(i) = a_out(i-1) + a_out(i-2)
    end if
  end do
end subroutine fibonacci
```

You can try this on your laptop, or on ARCHER.

In [ ]:
!ls

<br>
<hr class="top">
## Archer
<hr class="bot">
<br>

Consult the sheet with your guest account details to login.

```
$ cd /work/y14/y14/guestNNN
$ wget https://www.archer.ac.uk/training/course-material/2018/09/scipy-durham/sci-py-archer-course.tar.gz
$ tar zxvf sci-py-archer-course.tar.gz

```
and

```
$ cd sci-py-archer-course/lectures/lecture05-couple
```
You will need an editor to edit files.

<br>
<hr class="top">
## Archer 
<hr class="bot">
<br>

From the command line
```bash
bash> module swap PrgEnv-cray PrgEnv-gnu
bash> module swap gcc gcc/4.9.3
bash> module load anaconda
```

Use the commands from the cells above on the command line.

Create a script or use the ipython shell to run on the front end
<br>
<br>

<br>
<hr class="top">
## C Exercise
<hr class="bot">
<br>

An equivalent C function is available to compute a Fibonacci series: `fibonacci.c`

Try using `ctypes` to call this function ither using your laptop, or Archer.

If you are using your laptop, your will need a C compiler; on Archer, use `cc`.

<hr class="top">
<hr class="bot">


In [1]:
# This cell is for the presenter
from IPython.core.display import display, HTML

styles = open("../style.css", "r").read()
display(HTML(styles))